In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Thesis Papers/SMHD/final_cleaned_data.csv")

In [ ]:
df["Label"].value_counts()

control          15202
depression       14139
bipolar           6434
ptsd              2894
schizophrenia     1331
Name: Label, dtype: int64

In [ ]:
df_c = df[df["Label"] == "control"]
df_d = df[df["Label"] == "schizophrenia"]
df_c = df_c.sample(frac =1 )
df_c = df_c.sample(frac =1 ).iloc[0:1500,]
df_d = df_d.sample(frac =1 )
# df_d = df_d.sample(frac =1 ).iloc[0:5000,]

In [ ]:
def stripstr(x):
   return len(x.strip())==0

In [ ]:
result = pd.concat([df_c, df_d], ignore_index=True, sort=False)

In [ ]:
empty = result[result["Posts"].apply(stripstr)]

In [ ]:
indexarr = empty.index
result.drop(index = indexarr, inplace=True)

In [ ]:
result = result.sample(frac = 1)

In [ ]:
# def cut(sen, length = 1000):
#   content = sen.split()
#   if len(content) < length:
#     return " ".join(content)
#   return " ".join(content[0:length])

In [ ]:
# x=newdf['Posts'].apply(cut)

In [ ]:
result["Label"].value_counts()

control          1491
schizophrenia    1330
Name: Label, dtype: int64

In [ ]:
x=result["Posts"]
y=result["Label"]

In [ ]:
y=result["Label"]

In [ ]:
y = np.array(list(map(lambda x: 1 if x=="schizophrenia" else 0, y)))

# bert

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 51kB 5.5MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=6f023c9d9ae50f39328f4753fb0e5485775cfc7674fb3f18a1162065f298a8ab
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=439e964285c571fe865fecb676c21204f7b7d8f4475a5bfe7dfbe5956ed963c1
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=10384f57ff5bd7643ccd8a71297a75de049233b81f46ee7d744072457a27d3a8
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 7.6MB/s 


In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
def tokenize_reviews(text_reviews):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [ ]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [ ]:
type(tokenized_reviews[0])

list

In [ ]:
reviews_with_len = []
i=0
for  review in tokenized_reviews:
  reviews_with_len.append([review, y[i], len(review)])
  i=i+1


In [ ]:
reviews_with_len.sort(key=lambda x: x[2])

In [ ]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 491), dtype=int32, numpy=
 array([[ 6429,     0,     0, ...,     0,     0,     0],
        [ 2204,  2678,     0, ...,     0,     0,     0],
        [ 1054,  6806,  2050, ...,     0,     0,     0],
        ...,
        [ 8040, 29194,  3501, ...,     0,     0,     0],
        [ 2130,  2488,  2084, ...,     0,     0,     0],
        [ 3718,  2031,  2000, ...,  2096,  2017,  2064]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)>)

In [ ]:
import math

In [ ]:
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
  def __init__(self,vocabulary_size,embedding_dimensions=128,cnn_filters=50, dnn_units=512, model_output_classes=2, dropout_rate=0.1, training=False,name="text_model"):
    super(TEXT_MODEL, self).__init__(name=name)
    
    self.embedding = layers.Embedding(vocabulary_size,embedding_dimensions)
    self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,kernel_size=2,padding="valid",activation="relu")
    self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,kernel_size=3,padding="valid",activation="relu")
    self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,kernel_size=4,padding="valid",activation="relu")
    self.pool = layers.GlobalMaxPool1D()
    
    self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
    self.dropout = layers.Dropout(rate=dropout_rate)
    if model_output_classes == 2:
      self.last_dense = layers.Dense(units=1, activation="sigmoid")
    else:
      self.last_dense = layers.Dense(units=model_output_classes, activation="softmax")
    
  def call(self, inputs, training):
    l = self.embedding(inputs)
    l_1 = self.cnn_layer1(l) 
    l_1 = self.pool(l_1) 
    l_2 = self.cnn_layer2(l) 
    l_2 = self.pool(l_2)
    l_3 = self.cnn_layer3(l)
    l_3 = self.pool(l_3) 
    
    concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
    concatenated = self.dense_1(concatenated)
    concatenated = self.dropout(concatenated, training)
    model_output = self.last_dense(concatenated)
    
    return model_output

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [ ]:
if OUTPUT_CLASSES == 2:
  text_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
else:
  text_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["sparse_categorical_accuracy"])

In [ ]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
     79/Unknown - 170s 2s/step - loss: 0.6540 - accuracy: 0.6217